In [2]:
import requests
import urllib.request 
from urllib.request import urlopen 
from bs4 import BeautifulSoup
import re
import json
import pandas as pd
import numpy as np
from numpy.random import randint 
from datetime import date
import datetime as dt
import os 
import glob

In [3]:
rootpage = 'https://www.rottentomatoes.com/sitemap_' 
end = '.xml'
today = date.today()

def getLinks(pageUrl):
    global pages 
    for sitemap_page in range(0, 25): 
        pages = set() 
        html = urlopen(rootpage+str(sitemap_page)+end)
        bsObj = BeautifulSoup(html)
        
        for link in bsObj.findAll('loc'):
            replacers = ['trailers/', 'pictures/']
            link = str(link).replace('</loc>', '').split('.com')
            
            if link[1] not in pages: 
                pages.add(link[1])
    
        pages = pd.DataFrame(pages, columns=['links'])
        pages.to_csv(r'/Users/mimiyufanyou/Documents/GitHub/rottentomatoes/movie_links/links_'+str(today)+'_'+str(sitemap_page)+'.csv', sep=',')
        
        print('saved to csv:','links_'+str(today)+'_'+str(sitemap_page)+'.csv', dt.datetime.now())

getLinks('')
        

saved to csv: links_2019-11-14_0.csv 2019-11-14 21:39:11.174195
saved to csv: links_2019-11-14_1.csv 2019-11-14 21:39:12.712967
saved to csv: links_2019-11-14_2.csv 2019-11-14 21:39:14.400965
saved to csv: links_2019-11-14_3.csv 2019-11-14 21:39:15.970907
saved to csv: links_2019-11-14_4.csv 2019-11-14 21:39:17.629636
saved to csv: links_2019-11-14_5.csv 2019-11-14 21:39:18.747092
saved to csv: links_2019-11-14_6.csv 2019-11-14 21:39:20.373582
saved to csv: links_2019-11-14_7.csv 2019-11-14 21:39:22.043285
saved to csv: links_2019-11-14_8.csv 2019-11-14 21:39:24.153221
saved to csv: links_2019-11-14_9.csv 2019-11-14 21:39:26.594706
saved to csv: links_2019-11-14_10.csv 2019-11-14 21:39:28.104244
saved to csv: links_2019-11-14_11.csv 2019-11-14 21:39:29.387612
saved to csv: links_2019-11-14_12.csv 2019-11-14 21:39:30.994940
saved to csv: links_2019-11-14_13.csv 2019-11-14 21:39:32.511277
saved to csv: links_2019-11-14_14.csv 2019-11-14 21:39:34.220567
saved to csv: links_2019-11-14_15.c

In [4]:
filepath = r'/Users/mimiyufanyou/Documents/GitHub/rottentomatoes/movie_links/*.csv'
df_links = pd.concat([pd.read_csv(f) for f in glob.glob(filepath)], ignore_index = True)
df_links = list(df_links['links'])

In [5]:
trimmed_links = set()

for links in df_links:
    links = links.replace('/pictures/', '').replace('/trailers/', '') 
    trimmed_links.add(links)

In [12]:
baseurl = 'https://www.rottentomatoes.com'
listlength = len(trimmed_links)
movie_df = pd.DataFrame(columns=['url', 'title', 'tomatoScore', 'popcornScore'])
today = date.today()

def getScores(link):
    try: 
        siteurl = urlopen(baseurl+link)

        bsObj = BeautifulSoup(siteurl)
        movie_title = str(bsObj).split('"name":"')[1].split('"')[0]
        #print(movie_title)

        tomatoScore = str(bsObj).split('ratingValue":')[1].split(',"')[0]
        #print(tomatoScore)

        ems_id = str(bsObj).split('emsId":"')[1].split('"')[0]
        ems_req_url = f'https://www.rottentomatoes.com/napi/audienceScore/{ems_id}'

        ems_response = requests.get(ems_req_url)
        ems_soup = BeautifulSoup(ems_response.text, 'html.parser')
        ems_text_response = ems_response.text
        json_ems = json.loads(ems_text_response)
        popcornScore = json_ems['audienceScoreAll']['score']
        #print(popcornScore)

        movie_df.loc[link] = [link, movie_title, tomatoScore, popcornScore]
    except: 
        pass

In [13]:
df_links = pd.DataFrame(df_links)
df_links.iloc[0:2]

,0
0,/m/quality_street
1,/m/1028589-johnny_come_lately/pictures/


In [17]:
for batch in range(0,5):
    for link in df_links.iloc[batch]: 
        movie_name = df_links.iloc[batch][0]
        getScores(movie_name)

movie_df.to_csv(r'/Users/mimiyufanyou/Documents/GitHub/rottentomatoes/movie_scores/scores_'+str(today)+'_'+str(batch)+'.csv', sep=',')
print('saved to csv:','scores_'+str(today)+'_'+str(batch)+'.csv', dt.datetime.now())

saved to csv: scores_2019-11-14_4.csv 2019-11-14 21:44:58.895871


In [19]:
replacetextnulls = lambda x: x.replace('null', np.nan)
movie_df_copy = movie_df.apply(replacetextnulls).dropna(axis=0, how='any')

movie_df_copy['squidScore'] = movie_df_copy['popcornScore'].astype(int)-movie_df_copy['tomatoScore'].astype(int)
movie_df_copy.sort_values(by=['squidScore'], ascending=False)

,url,title,tomatoScore,popcornScore,squidScore
/m/iron_man_2,/m/iron_man_2,Iron Man 2,73,71,-2
/m/1212111-triangle,/m/1212111-triangle,Triangle,80,66,-14
/m/quality_street,/m/quality_street,Quality Street,67,31,-36


In [21]:
movies = pd.DataFrame(movie_df_copy)
movies.to_csv(r'/Users/mimiyufanyou/Documents/GitHub/rottentomatoes/movie_final/scores'+str(today)+'.csv', sep=',')